In [1]:
import torch
print(torch.cuda.is_available()) 
print(torch.cuda.get_device_name(0))  # GPU name

True
NVIDIA GeForce GTX 1650


In [2]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
import ollama
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import pipeline

c:\Users\dp971\Coading\Capstone\RAG\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# loader = PyPDFLoader("./doc/budget-2024.pdf")
# doc = loader.load()
# type(doc)

list

In [ ]:
# text_spliter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 30, separator = "\n")
# docs = text_spliter.split_documents(documents = doc)


[Document(metadata={'source': './doc/budget-2024.pdf', 'page': 0}, page_content=''), Document(metadata={'source': './doc/budget-2024.pdf', 'page': 1}, page_content='©His Majesty the King in Right of Canada (2024) \nAll rights reserved\nAll requests for permission to reproduce this document \nor any part thereof shall be addressed to the Department of Finance Canada.\nThis document is available at www.Canada.ca/Budget\nCette publication est aussi disponible en français.\nF1-24/3E-PDF\nCat No.: 1719-7740\nIn case of discrepancy between the printed version and the electronic version, \nthe electronic version will prevail.'), Document(metadata={'source': './doc/budget-2024.pdf', 'page': 2}, page_content='   i \nDeputy Prime Minister’s Foreword \nA fair chance to build a good middle class life—to do as well as your parents, or \nbetter—that’s the promise of Canada. For too many, especially for younger \nCanadians, that promise is at risk.  \nWe have a plan to fix that. We have a plan to bui

In [4]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"   # It maps sentences & paragraphs to a 384 dimensional dense vector space
model_kwargs = {'device': 'cuda'} # it will use googl cloud gpu..
embeddings = HuggingFaceEmbeddings(model_name=embedding_model, model_kwargs=model_kwargs) #model_kwargs=model_kwargs

C:\Users\dp971\AppData\Local\Temp\ipykernel_11612\127585856.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model, model_kwargs=model_kwargs) #model_kwargs=model_kwargs


In [ ]:
#vectorstore = FAISS.from_documents(docs, embeddings)

In [ ]:
#vectorstore.save_local("faiss_index")

In [5]:
#loading embedding...
embedding_vectorestore = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)

In [6]:
retriever = embedding_vectorestore.as_retriever(search_type="similarity")

curl -fsSL https://ollama.ai/install.sh | sh

ollama serve & ollama pull llama3.2:1b

In [11]:
from langchain_ollama import OllamaLLM
llm = OllamaLLM(model="llama3.2:1b")

In [12]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [13]:
result = qa.invoke("What is the current unemployment rate in Canada?")
print(result)

{'query': 'What is the current unemployment rate in Canada?', 'result': 'The current unemployment rate in Canada is 6.1% as of March 2024, according to the latest data from Statistics Canada.'}


In [23]:
result = qa.invoke("How much support does the Child Disability Benefit provide on average annually?")
print(result)

{'query': 'How much support does the Child Disability Benefit provide on average annually?', 'result': 'I don\'t know, because it\'s not provided in the text. The text only mentions that the Child Disability Benefit provides an "average of approximately $3,000 in annual support" but doesn\'t give a specific breakdown or amount per year.'}


In [24]:
result = qa.invoke("How much is Saskatchewan investing to expand its capacity to assess international medical graduates?")
print(result)

{'query': 'How much is Saskatchewan investing to expand its capacity to assess international medical graduates?', 'result': "According to the text, Alberta's Mount Royal University is creating 256 new seats for the Bridge to Canadian Nursing program to support internationally educated nurses as they transition into the Canadian healthcare system."}


In [25]:
result = qa.invoke("How much funding did Quebec announce for health care over four years in March 2024?")
print(result)

{'query': 'How much funding did Quebec announce for health care over four years in March 2024?', 'result': 'Quebec announced $2 billion over four years.'}


In [26]:
result = qa.invoke("What is my dogs name?")
print(result)

{'query': 'What is my dogs name?', 'result': 'I don\'t know the name of your dog. The text only provides information about various government agencies and their funding proposals, but it does not mention a specific person or pet named "dog".'}


In [15]:
import torch
torch.cuda.empty_cache()
